In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import tensorflow  as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

from tensorflow.keras.callbacks import *

from sklearn.metrics import mean_absolute_error
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

import os

In [21]:
frame = pd.read_csv(r'ausleihe.csv')
df = pd.DataFrame(frame)
col = list(df.columns.values)

df.head()

,Year,Week,LAT,LONG,count
0,2015,1,40.680342,-73.955769,18
1,2015,1,40.680983,-73.950048,18
2,2015,1,40.682166,-73.953990,8
3,2015,1,40.682232,-73.961458,19
4,2015,1,40.683125,-73.978951,59


In [22]:
#get input and output
x = df.drop("count",axis=1)
y = df["count"]

In [23]:
#split data into train and test
x_train, x_test ,y_train , y_test = train_test_split(x,y,test_size=0.2,shuffle=False)

In [36]:
#normalise data
ct = make_column_transformer(
 (MinMaxScaler(),["Year","Week","LAT","LONG"]) , remainder="passthrough"
 )

ct.fit(x_train)

x_train_normalized = ct.transform(x_train)

x_test_normalized = ct.transform(x_test)

x_train_normalized.shape

(116651, 4)

In [42]:
#create Model structure
with tf.device('/device:CPU:0'):
    model = Sequential()
    model.add(Dense(65, kernel_initializer='normal',  activation='relu', input_shape=(4,) ))    
    model.add(Dense(35, kernel_initializer='normal',activation='relu'))   
    model.add(Dense(35, kernel_initializer='normal',activation='relu'))   
    model.add(Dense(1, kernel_initializer='normal',activation=None))

In [38]:
gpus = tf.config.experimental.list_physical_devices('CPU')
print(gpus)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [44]:
#set model loss and optimizer
with tf.device('/device:CPU:0'):
    model.compile(optimizer= Adam(learning_rate=0.001),loss='mae', metrics=['mae','mape'])

In [45]:
#checkpoint to save best Model which isnt overfitted
checkpoint = ModelCheckpoint('Model1', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='min' )

In [46]:
#train Model
with tf.device('/device:CPU:0'):
    history = model.fit(x_train_normalized, y_train.values, epochs=100 , batch_size=1,shuffle=False   )

Epoch 1/100
116651/116651 [==============================] - 113s 968us/step - loss: 288.0895 - mae: 288.0895 - mape: 243.9989
Epoch 2/100
116651/116651 [==============================] - 108s 928us/step - loss: 267.7480 - mae: 267.7480 - mape: 233.3448
Epoch 3/100
116651/116651 [==============================] - 113s 972us/step - loss: 213.7521 - mae: 213.7521 - mape: 201.5468
Epoch 4/100
116651/116651 [==============================] - 106s 906us/step - loss: 212.4809 - mae: 212.4809 - mape: 205.2515
Epoch 5/100
116651/116651 [==============================] - 107s 919us/step - loss: 221.0622 - mae: 221.0622 - mape: 261.9853
Epoch 6/100
116651/116651 [==============================] - 124s 1ms/step - loss: 215.2264 - mae: 215.2264 - mape: 208.3204
Epoch 7/100
116651/116651 [==============================] - 111s 952us/step - loss: 210.8884 - mae: 210.8884 - mape: 201.9202
Epoch 8/100
116651/116651 [==============================] - 130s 1ms/step - loss: 207.9151 - mae: 207.9151 - map

KeyboardInterrupt: 

In [47]:
model.evaluate(x_test_normalized,y_test.values)

912/912 [==============================] - 1s 733us/step - loss: 319.1625 - mae: 319.1625 - mape: 123.8683


[319.1625061035156, 319.1625061035156, 123.8683090209961]